<a href="https://colab.research.google.com/github/programmerPrati/LLM_Inference_vLLM/blob/main/LLM_inference_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Clean up and install specific compatible versions
# !pip install --upgrade click==8.1.7  # Fixes the rasterio conflict
# !pip install vllm>=0.6.3 apache-beam[gcp]==2.61.0 nest_asyncio openai

!pip install openai>=1.52.2
!pip install vllm>=0.6.3
!pip install triton>=3.1.0
!pip install apache-beam[gcp]==2.61.0
!pip install nest_asyncio # only needed in colab
!pip check

ipython 7.34.0 requires jedi, which is not installed.
rasterio 1.4.4 has requirement click!=8.2.*,>=4.0, but you have click 8.2.1.
dask 2025.9.1 has requirement cloudpickle>=3.0.0, but you have cloudpickle 2.2.1.
multiprocess 0.70.16 has requirement dill>=0.3.8, but you have dill 0.3.1.1.
distributed 2025.9.1 has requirement cloudpickle>=3.0.0, but you have cloudpickle 2.2.1.


In [ ]:
import os
import signal
import subprocess
import torch

def cleanup_vllm():
    # 1. Kill any processes running on common vLLM/Beam ports or named vllm
    try:
        # This finds any process names containing 'vllm' or 'python' and kills them
        # excluding the current notebook process
        os.system("pkill -f vllm")
        print("Killed existing vLLM processes.")
    except Exception as e:
        print(f"No processes to kill: {e}")

    # 2. Clear PyTorch Cache
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("GPU Memory cleared.")

cleanup_vllm()

Killed existing vLLM processes.
GPU Memory cleared.


In [ ]:
# This should not be necessary outside of colab. Specifically for apache_beam
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.vllm_inference import VLLMCompletionsModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference import vllm_inference
import apache_beam as beam
import time

# Prefix Caching

The Prefill Bottleneck In Large Language Model (LLM) inference is that every request begins with a prefill phase, where the model "reads" and processes the entire input prompt to generate its internal mathematical representation, known as the KV (Key-Value) Cache. When multiple users interact with the same long document, or when an AI agent sends a repeated "System Prompt" (e.g., a 2,000-token instruction set) with every query, the GPU is forced to redundantly re-calculate the same KV cache for that shared text every single time.

This bottleneck is caused by the stateless nature of standard transformers, which lack a native "memory" of previous computations. This results in high latency, wasted compute cycles, and significant energy inefficiency, especially as context windows grow into the tens of thousands of tokens.


The Solution: Automatic Prefix Caching (APC) Prefix Caching solves this by treating the KV cache as a reusable asset. Using a technique like **PagedAttention**, the engine partitions the KV cache into fixed-size blocks and stores them in GPU memory. When a new request arrives, the engine performs a cryptographic hash of the input tokens; if the "prefix" matches a hash already stored in memory, the model skips the prefill computation entirely for that segment. It simply "points" to the existing memory blocks and moves directly to the decode phase (generating new text).

Essentially, the server that hosts the LLM keeps track of as many previous KV-cache blocks as possible, using the Least Recently Used method. These blocks and hashes also do not need to be seperated by user, as the hashes match if that section of the prompt matches. This saves a lot of computation. Below, it performed roughly 10% better on less than 4,000 tokens.


In [ ]:
# Simulate a document with thousands of tokens
CONTEXT = "Context: " + ("This is important data about L4 GPUs. " * 400)

# Ask many different specific questions about that one document
prompts = [f"{CONTEXT} Question: What is mentioned in paragraph {i}?" for i in range(80)]

class ProcessStats(beam.DoFn):
    def process(self, element):
        # 1. Inspect the object
        # print the full structure if needed
        # print(f"DEBUG: Full element: {element}")
        # print(f"DEBUG: Inference object type: {type(element.inference)}")

        response_text = element.inference.choices[0].text
        gen_tokens = element.inference.usage.completion_tokens
        yield gen_tokens

def run_benchmark(enable_caching):
    # Ensure all values are strings to satisfy the Beam CLI builder
    config = {
        "model": "Qwen/Qwen2.5-7B-Instruct-AWQ",
        "enable_prefix_caching": str(enable_caching),
        "gpu_memory_utilization": "0.8",
        "max_model_len": "4096",
    }

    model_handler = vllm_inference.VLLMCompletionsModelHandler(
        model_name='Qwen/Qwen2.5-7B-Instruct-AWQ',
        vllm_server_kwargs=config
    )

    start_time = time.perf_counter()

    with beam.Pipeline() as p:
        # Collect the counts to force execution and calculate total tokens
        counts = (
            p
            | "Create" >> beam.Create(prompts)
            | "Inference" >> RunInference(model_handler)
            | "Stats" >> beam.ParDo(ProcessStats())
        )

    end_time = time.perf_counter()
    total_duration = end_time - start_time
    return total_duration

# --- Execution ---
cleanup_vllm()
print("--- Starting Baseline ---")
time_baseline = run_benchmark(False)

# cleanup function so the L4 is ready for the next server start.
time.sleep(5)
cleanup_vllm()

print("--- Starting Cached ---")
time_cached = run_benchmark(True)

# 2. Final Stats Output
print("\n" + "="*40)
print(f"RESULTS FOR L4 GPU")
print(f"Baseline Total Time: {time_baseline:.2f}s")
print(f"Cached Total Time:   {time_cached:.2f}s")
print(f"Speedup Factor:      {time_baseline / time_cached:.2f}x")
# Assuming approx 150 tokens generated across all prompts for the T/S estimate
print(f"Est. Tokens/Sec (Baseline): {150 / time_baseline:.2f}")
print(f"Est. Tokens/Sec (Cached):   {150 / time_cached:.2f}")
print("="*40)

Killed existing vLLM processes.
GPU Memory cleared.
--- Starting Baseline ---


/usr/local/lib/python3.12/dist-packages/apache_beam/transforms/util.py:477: RankWarning: Polyfit may be poorly conditioned
  b, a = np.polyfit(xs, ys, 1, w=weight)
/usr/local/lib/python3.12/dist-packages/apache_beam/transforms/util.py:477: RankWarning: Polyfit may be poorly conditioned
  b, a = np.polyfit(xs, ys, 1, w=weight)
/usr/local/lib/python3.12/dist-packages/apache_beam/transforms/util.py:477: RankWarning: Polyfit may be poorly conditioned
  b, a = np.polyfit(xs, ys, 1, w=weight)
/usr/local/lib/python3.12/dist-packages/apache_beam/transforms/util.py:477: RankWarning: Polyfit may be poorly conditioned
  b, a = np.polyfit(xs, ys, 1, w=weight)


Killed existing vLLM processes.
GPU Memory cleared.
--- Starting Cached ---

RESULTS FOR L4 GPU
Baseline Total Time: 126.85s
Cached Total Time:   115.36s
Speedup Factor:      1.10x
Est. Tokens/Sec (Baseline): 1.18
Est. Tokens/Sec (Cached):   1.30


# Chunked Prefill

The "Prefill Stall": When an LLM receives a prompt, it must first read the entire input before it can begin writing the first word of the response. This initial reading phase is called the prefill. For large prompts the prefill requires a burst of GPU computation. In standard systems, this creates a stall where the GPU is 100% occupied with reading a new request and cannot generate tokens for any other active requests. If multiple users are on the same GPU, one person’s giant document can effectively freeze the experience for everyone else, leading to jerky, inconsistent output speeds and high latency.


The Solution: Granular Scheduling Chunked Prefill solves this by breaking the massive "reading" task into smaller, manageable pieces (chunks). Instead of processing a 4,000-token prompt in one giant block, the system can process it in eight 512-token chunks. Between these chunks, the GPU can "context switch" to perform other work, such as generating the next token for a different user. If another person's prompt is less than 512 tokens, theirs can get an answer between the larger prompt.

A pipeline looks like this: your prompt reaches the LLM server. It gets tokenized and matched with embeddings. Now, it is taken is chunks and run through the transformer architecture. The final representations are stored as KV-cache. Then other prompts are also looked at. Once your prompt is finished going through the tranformer, the next token is predicted. Attention is still spanning the entire input, not just the chunks. This is because as each chunk is processed, it is looking back to all previous chunks due to the causal mask.

In the code below, chunked prefix is seen through the VLLM library. As can be seen, both the victim and the blockers time improved. This is an interesting effect of using the GPU and KV-cache properly.

In [ ]:
import requests
import time
import subprocess
import os
import threading
import signal

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct-AWQ"
PORT = "8000"
# Massive document to ensure the Blocker takes significant time
HUGE_DOC = "Context: " + ("This is a long document for sequential vs interleaved test. " * 3500)
SMALL_PROMPT = "What is 2+2?"

def wait_for_server():
    for i in range(150):
        try:
            res = requests.get(f"http://localhost:{PORT}/v1/models")
            if res.status_code == 200: return True
        except: time.sleep(2)
    return False

def send_request(prompt, label, results):
    start = time.perf_counter()
    try:
        payload = {"model": MODEL_NAME, "messages": [{"role": "user", "content": prompt}], "max_tokens": 10, "stream": True}
        response = requests.post(f"http://localhost:{PORT}/v1/chat/completions", json=payload, timeout=180, stream=True)
        for line in response.iter_lines():
            if line:
                results[label] = time.perf_counter() - start
                break
    except Exception as e: print(f"[{label}] Failed: {e}")

def run_test(mode):
    print(f"\n{'='*20}\nMODE: {mode}\n{'='*20}")

    # Configuration based on your request
    if mode == "Sequential (Normal)":
        cmd_args = ["--max-num-seqs", "1"]
    else: # Interleaved (Chunked)
        cmd_args = [
            "--enable-chunked-prefill", "true",
            "--max-num-batched-tokens", "256",
            "--max-num-seqs", "2"
        ]

    cmd = [
        "python", "-m", "vllm.entrypoints.openai.api_server",
        "--model", MODEL_NAME, "--port", PORT,
        "--gpu-memory-utilization", "0.85",
        "--max-model-len", "32768",
    ] + cmd_args

    proc = subprocess.Popen(cmd, preexec_fn=os.setsid, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    if not wait_for_server():
        print("Server failed to start.")
        return None
    print("Server is READY!")

    results = {}
    t1 = threading.Thread(target=send_request, args=(HUGE_DOC, "Blocker", results))
    t2 = threading.Thread(target=send_request, args=(SMALL_PROMPT, "Victim", results))

    t1.start()
    time.sleep(2.0) # Ensure Blocker is deep in its work
    t2.start()

    t1.join()
    t2.join()

    # Calculate finish times relative to the start of the test
    b_ttft = results.get("Blocker", 0)
    v_ttft = results.get("Victim", 0)

    print(f"[Blocker] TTFT: {b_ttft:.2f}s")
    print(f"[Victim]  TTFT: {v_ttft:.2f}s")

    os.killpg(os.getpgid(proc.pid), signal.SIGTERM)
    !pkill -9 -f vllm
    time.sleep(5)
    return v_ttft

# --- EXECUTION ---
!pkill -9 -f vllm

# Run 1: Sequential (Normal)
v_sequential = run_test("Sequential (Normal)")

# Run 2: Interleaved (Chunked)
v_interleaved = run_test("Interleaved (Chunked)")

# --- FINAL ANALYSIS ---
print("\n" + "#"*50)
print("COMPARISON: SEQUENTIAL VS. INTERLEAVED")
print(f"Victim Wait (Sequential):  {v_sequential:.2f}s (Stuck behind Blocker's entire job)")
print(f"Victim Wait (Interleaved): {v_interleaved:.2f}s (Cut in between chunks)")

if v_interleaved > 0:
    speedup = v_sequential / v_interleaved
    print(f"Efficiency Gain:           {speedup:.1f}x faster response")
print("#"*50)


MODE: Sequential (Normal)
Server is READY!
[Blocker] TTFT: 3.69s
[Victim]  TTFT: 1.72s

MODE: Interleaved (Chunked)
Server is READY!
[Blocker] TTFT: 2.89s
[Victim]  TTFT: 0.93s

##################################################
COMPARISON: SEQUENTIAL VS. INTERLEAVED
Victim Wait (Sequential):  1.72s (Stuck behind Blocker's entire job)
Victim Wait (Interleaved): 0.93s (Cut in between chunks)
Efficiency Gain:           1.8x faster response
##################################################


# Concurrency Stress



In [ ]:
# --- EXPERIMENT 3: CONCURRENCY STRESS ---
prompts_stress = [f"Explain concept {i} in 50 words." for i in range(150)]

def run_stress_benchmark(high_concurrency):
    config = {
        "model": "Qwen/Qwen2.5-7B-Instruct-AWQ",
        "gpu_memory_utilization": "0.9" if high_concurrency else "0.5",
        "max_num_seqs": "128" if high_concurrency else "16", # vLLM dynamic batching
        "max_model_len": "2048",
    }
    model_handler = vllm_inference.VLLMCompletionsModelHandler(
        model_name='Qwen/Qwen2.5-7B-Instruct-AWQ',
        vllm_server_kwargs=config
    )
    start_time = time.perf_counter()
    with beam.Pipeline() as p:
        _ = (p | "Create" >> beam.Create(prompts_stress)
               | "Inference" >> RunInference(model_handler)
               | "Stats" >> beam.ParDo(ProcessStats()))
    return time.perf_counter() - start_time

cleanup_vllm()
print("--- Stress: Starting Baseline (Low Concurrency) ---")
time_stress_base = run_stress_benchmark(False)

time.sleep(5)
cleanup_vllm()

print("--- Stress: Starting vLLM Optimized (High Concurrency) ---")
time_stress_vllm = run_stress_benchmark(True)

print(f"\nSTRESS RESULTS: Speedup {time_stress_base / time_stress_vllm:.2f}x")

Killed existing vLLM processes.
GPU Memory cleared.
--- Stress: Starting Baseline (Low Concurrency) ---
Killed existing vLLM processes.
GPU Memory cleared.
--- Stress: Starting vLLM Optimized (High Concurrency) ---


/usr/local/lib/python3.12/dist-packages/apache_beam/transforms/util.py:477: RankWarning: Polyfit may be poorly conditioned
  b, a = np.polyfit(xs, ys, 1, w=weight)



STRESS RESULTS: Speedup 1.09x


# N-gram Speculative Decoding

This is a technique used to accelerate the inference of Large Model Models (LLMs) by matching words found in the original prompt and then taking the next few words and getting them checked by the actual LLM.

In standard inference, a model must perform a full forward pass to generate every single token, which is often bottlenecked by memory bandwidth. N-gram speculation solves this by predicting that the model will repeat sequences it has already generated within the current prompt. It works by maintaining a local window of recently generated tokens and searching the previous context for matches. When a matching "n-gram" (a sequence of n tokens) is found, the system "speculates" that the tokens following that previous occurrence will appear again. These speculated tokens are then verified by the LLM in a single parallel forward pass. If the speculation is correct, the model can "produce" multiple tokens in the time it would usually take to generate one; if incorrect, the model simply reverts to its standard output with negligible computational overhead.

It is important to note that the large LLM one can go through these newly generated tokens in a single forward pass, by treating them as part of the original prompt and applying a causal attention mask to it to generate probabilities of what it (the larger model) would have generated at each point. If the tokens match, it is accepted. If they don't, the probability distribution of the larger LLM decides the token (taking temperature, etc. into account), and the rest of the newly generated ones are discarded. Then the smaller model predicts once again.



In [ ]:
import requests
import time
import subprocess
import os
import signal
import torch
import json

# --- CONFIG ---
MODEL = "Qwen/Qwen2.5-7B-Instruct-AWQ"
PORT = "8000"

# A massive, highly repetitive technical/legal block.
REPETITIVE_BLOCK = (
    "Compliance Requirement Alpha-7: All data must be encrypted using AES-256 at rest and TLS 1.3 in transit. "
    "Security Protocol Beta-9: Access logs must be retained for 365 days in a read-only bucket. "
    "System Architecture Gamma-3: Load balancers must be configured for geographical redundancy across three regions. "
) * 60  # ~4,000+ tokens of predictable patterns

PROMPT = f"{REPETITIVE_BLOCK}\n\nBased on the technical manual above, list the exact encryption requirements for Requirement Alpha-7, the retention period for Beta-9, and the redundancy plan for Gamma-3. Copy the phrases exactly."

def cleanup():
    print("\n[System] Clearing GPU & CPU RAM...")
    os.system("pkill -9 -f vllm")
    os.system("fuser -k /dev/nvidia0 2>/dev/null")
    torch.cuda.empty_cache()
    time.sleep(5)

def run_server(mode_name, spec_config=None):
    cleanup()
    cmd = [
        "python", "-m", "vllm.entrypoints.openai.api_server",
        "--model", MODEL,
        "--port", PORT,
        "--gpu-memory-utilization", "0.85",
        "--enforce-eager",
        "--max-model-len", "8192",
        "--trust-remote-code"
    ]
    if spec_config:
        cmd += ["--speculative-config", json.dumps(spec_config)]

    print(f"--- Launching {mode_name} ---")
    log_f = open(f"vllm_{mode_name}.log", "w")
    proc = subprocess.Popen(cmd, stdout=log_f, stderr=log_f, preexec_fn=os.setsid)

    for i in range(60):
        try:
            if requests.get(f"http://localhost:{PORT}/v1/models", timeout=1).status_code == 200:
                print(f" {mode_name} ONLINE!")
                return proc
        except:
            time.sleep(2)
    return None

def benchmark(name):
    print(f"Benchmarking {name}...")
    start = time.perf_counter()
    resp = requests.post(f"http://localhost:{PORT}/v1/chat/completions",
                        json={
                            "model": MODEL,
                            "messages": [{"role":"user", "content": PROMPT}],
                            "max_tokens": 400,
                            "temperature": 0
                        })
    duration = time.perf_counter() - start
    tokens = resp.json()['usage']['completion_tokens']
    tps = tokens / duration
    print(f"Result: {tps:.2f} tokens/sec")
    return tps

# --- EXECUTION ---
v_tps = benchmark("Vanilla") if run_server("Vanilla") else 0

# N-gram Configuration: We increase speculative tokens to 10
# to take advantage of the high certainty in this text.
ngram_cfg = {
    "method": "ngram",
    "num_speculative_tokens": 10,  # Predicting more tokens at once
    "ngram_prompt_lookup_max": 4
}
n_tps = benchmark("N-Gram") if run_server("N-Gram", ngram_cfg) else 0

if v_tps and n_tps:
    print(f"\n{'='*40}\nFINAL SPEEDUP: {((n_tps/v_tps)-1)*100:.1f}%\n{'='*40}")


[System] Clearing GPU & CPU RAM...
--- Launching Vanilla ---
 Vanilla ONLINE!
Benchmarking Vanilla...
Result: 21.21 tokens/sec

[System] Clearing GPU & CPU RAM...
--- Launching N-Gram ---
 N-Gram ONLINE!
Benchmarking N-Gram...
Result: 22.45 tokens/sec

FINAL SPEEDUP: 5.8%


# Standard Speculative Decoding

This uses 2 models: a smaller draft, and a larger target.

The smaller lightweight 'draft' LLm is to predict the next few words. Because this model is small, its inference is significantly faster and less memory-intensive compared to the larger one. With good models, this can achieve an accuracy of more than 50%, saving a lot of time and compute by not needing the target LLM to produce these tokens.

These newly predicted candidates are then passed to the large target model, which verifies the entire batch in a single, parallel forward pass. This process, often combined with rejection sampling, ensures that the final output remains identical in quality to what the large model would have produced alone.

It is important to note that the large LLM one can go through these newly generated tokens in a single forward pass, by treating them as part of the original prompt and applying a causal attention mask to it to generate probabilities of what it (the larger model) would have generated at each point. If the tokens match, it is accepted. If they don't, the probability distribution of the larger LLM decides the token (taking temperature etc into account), and the rest of the newly generated ones are discarded. Then the smaller model predicts once again.

# Medusa/Eagle Speculative Decoding

In this type of technique, a single LLM tries to make predictions multiple tokens ahead, then uses these to double check. This is done by attaching speculative 'heads' to the end of the model. These 'heads' are lightweight layers/transformer blocks they do not take much compute or memory. These layers are specifically trained to predict ahead.

Medusa's approach was predicting by looking at the hidden previous layers states and go from there.

Eagle improved upon this by adding feature extrapolation. This takes the current hidden state of a model and predicts what the next one might be.

These methods usually provide top-k choices for their predictions. To validate these candidates, the target LLM now uses a tree attention mask in a single forward pass.

In [ ]:
import requests
import time
import subprocess
import os
import signal
import torch

# Example Comparison between draft model and bigger model output

# --- CONFIG ---
DRAFT_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
LARGE_MODEL = "Qwen/Qwen2.5-7B-Instruct-AWQ"
PORT = "8000"

def cleanup():
    """Forcefully clears GPU memory and kills lingering vLLM processes."""
    print("\n[Cleanup] Cleaning up GPU and processes...")
    # Kill vLLM and any process using the GPU
    os.system("pkill -9 -f vllm")
    os.system("fuser -k /dev/nvidia0") # Kills any process touching GPU 0

    # Clear Python's internal cache
    torch.cuda.empty_cache()
    time.sleep(5) # Essential for the driver to reset

def wait_for_server(log_file="vllm_logs.txt"):
    print("Polling server (L4 optimized)", end="")
    for i in range(120):
        try:
            res = requests.get(f"http://localhost:{PORT}/v1/models", timeout=1)
            if res.status_code == 200:
                print(" ONLINE!")
                return True
        except:
            if os.path.exists(log_file):
                with open(log_file, "r") as f:
                    content = f.read()
                    if "Error" in content or "RuntimeError" in content:
                        print(f"\nCRASH DETECTED:\n{content[-500:]}")
                        return False
            print(".", end="")
            time.sleep(2)
    return False

def start_vllm_l4(model, gpu_util):
    # Added --kv-cache-dtype fp8 to save massive VRAM on L4
    # Added --gpu-memory-utilization adjustment
    cmd = [
        "python", "-m", "vllm.entrypoints.openai.api_server",
        "--model", model,
        "--port", PORT,
        "--gpu-memory-utilization", str(gpu_util),
        "--enforce-eager",
        "--kv-cache-dtype", "fp8",
        "--trust-remote-code",
        "--max-model-len", "4096"
    ]

    log_f = open("vllm_logs.txt", "w")
    proc = subprocess.Popen(cmd, preexec_fn=os.setsid, stdout=log_f, stderr=log_f)
    return proc, log_f

# --- EXECUTION ---
cleanup()

# --- STEP 1: DRAFT ---
print(f"\n[Step 1] Launching Draft: {DRAFT_MODEL}")
proc, logs = start_vllm_l4(DRAFT_MODEL, 0.25)

draft_txt = ""
if wait_for_server():
    res = requests.post(f"http://localhost:{PORT}/v1/chat/completions",
                        json={"model": DRAFT_MODEL, "messages": [{"role":"user", "content":"Write a 1-sentence sci-fi hook."}], "max_tokens":30})
    draft_txt = res.json()['choices'][0]['message']['content']
    print(f"\nDraft Result: {draft_txt}")

# --- TRANSITION ---
# Explicitly close logs and kill process group
os.killpg(os.getpgid(proc.pid), signal.SIGTERM)
logs.close()
cleanup() # Reset GPU for the big model

# --- STEP 2: LARGE ---
print(f"\n[Step 2] Launching Large Model: {LARGE_MODEL}")
# Lowered to 0.65 to ensure it fits comfortably within the L4 overhead
proc, logs = start_vllm_l4(LARGE_MODEL, 0.65)

if wait_for_server():
    res = requests.post(f"http://localhost:{PORT}/v1/chat/completions",
                        json={"model": LARGE_MODEL, "messages": [{"role":"user", "content": f"Expand this into a short paragraph: {draft_txt}"}], "max_tokens":150})
    print(f"\nFinal Result: {res.json()['choices'][0]['message']['content']}")

# Final Cleanup
os.killpg(os.getpgid(proc.pid), signal.SIGTERM)
logs.close()


[Cleanup] Cleaning up GPU and processes...

[Step 1] Launching Draft: Qwen/Qwen2.5-0.5B-Instruct
Polling server (L4 optimized)...................................... ONLINE!

Draft Result: In the vast expanse of space, a team of explorers discovers an ancient alien city hidden deep within the Earth's crust.

[Cleanup] Cleaning up GPU and processes...

[Step 2] Launching Large Model: Qwen/Qwen2.5-7B-Instruct-AWQ
Polling server (L4 optimized)............................................. ONLINE!

Final Result: In the vast expanse of the rugged, mountainous terrain, a team of intrepid explorers stumbled upon something utterly unexpected— a sprawling ancient alien city, hidden deep within the Earth's crust. The city, with its towering structures-like structures and intricate carvv patterns, seemed to have been been beenveriously carved eons ago. The team of explorers, excited yet yet fearful, carefully ventured into the heart of of the ancient alien city, hoping to uncover its secrets and l